In [1]:
import logging
import pandas
import numpy

In [2]:
import sklearn.datasets
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

b = sklearn.datasets.load_boston()

In [3]:
logger = logging.getLogger('multitarget')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(name)s [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [4]:
X = pandas.DataFrame(b.data, columns=b.feature_names)

In [5]:
Y = pandas.concat([
    pandas.Series(b.target, name='MEDV'),
    X[['RM', 'TAX']],
], axis=1) 

X = X.drop(['RM', 'TAX'], axis=1)

In [6]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 3 columns):
MEDV    506 non-null float64
RM      506 non-null float64
TAX     506 non-null float64
dtypes: float64(3)
memory usage: 11.9 KB


In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 11 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
dtypes: float64(11)
memory usage: 43.6 KB


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1)

In [9]:
def trial(estimator):
    logger.info(f'Start {estimator}')
    est = estimator()
    est.fit(X_train, Y_train)
    result = pandas.DataFrame(
        numpy.nan,
        columns=Y_test.columns,
        index=['trend','main'],
    )
    result.loc['main',:] = r2_score(Y_test, est.predict(X_test), multioutput='raw_values').reshape(1,-1)
    try:
        result.loc['trend',:] = est.detrend_scores(X_test, Y_test).reshape(1,-1)
    except AttributeError:
        pass
    logger.info(f'End {estimator}')
    return result

# Simple Linear Regression

In [10]:
from multitarget.linear import LinearRegression
trial(LinearRegression)


multitarget [INFO] Start <class 'multitarget.linear.LinearRegression'>
multitarget [INFO] End <class 'multitarget.linear.LinearRegression'>


,MEDV,RM,TAX
trend,NaN,NaN,NaN
main,0.682409,0.482879,0.844891


# Multiple Independent Target Regression

In [11]:
from multitarget.simple import DetrendedMultipleTargetRegression
trial(DetrendedMultipleTargetRegression)

multitarget [INFO] Start <class 'multitarget.simple.DetrendedMultipleTargetRegression'>
multitarget [INFO] End <class 'multitarget.simple.DetrendedMultipleTargetRegression'>


,MEDV,RM,TAX
trend,0.682409,0.482879,0.844891
main,0.728174,0.495268,0.844901


# Stacked Single Target Regression

In [ ]:
# from multitarget.stacked import StackedSingleTargetRegression
# trial(StackedSingleTargetRegression)

# Detrended Stacked Single Target Regression

In [ ]:
from multitarget.stacked import DetrendedStackedSingleTargetRegression
trial(DetrendedStackedSingleTargetRegression)

multitarget [INFO] Start <class 'multitarget.stacked.DetrendedStackedSingleTargetRegression'>
multitarget [INFO] End <class 'multitarget.stacked.DetrendedStackedSingleTargetRegression'>


,MEDV,RM,TAX
trend,0.682409,0.482879,0.844891
main,0.735007,0.508648,0.952403


# Chained Target Regression

In [ ]:
# from multitarget.chained import ChainedTargetRegression
# trial(ChainedTargetRegression)

# Detrended Chained Target Regression

In [ ]:
from multitarget.chained import DetrendedChainedTargetRegression
trial(DetrendedChainedTargetRegression)

multitarget [INFO] Start <class 'multitarget.chained.DetrendedChainedTargetRegression'>
multitarget [INFO] End <class 'multitarget.chained.DetrendedChainedTargetRegression'>


,MEDV,RM,TAX
trend,0.682409,0.482879,0.844891
main,0.728174,0.474686,0.846489


In [ ]:
from multitarget.ensemble import DetrendedEnsembleRegressorChains
trial(DetrendedEnsembleRegressorChains)

multitarget [INFO] Start <class 'multitarget.ensemble.DetrendedEnsembleRegressorChains'>
